In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# torch.cuda.is_available()
# device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(device)
# torch.set_default_device(device)
device="cpu"

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load results

In [2]:
true_counts = torch.load("../images/setting2_true_counts.pt").to(device)
true_fluxes = torch.load("../images/setting2_true_fluxes.pt").to(device)
true_locs = torch.load("../images/setting2_true_locs.pt").to(device)
true_total_intensities = torch.load("../images/setting2_true_total_intensities.pt").to(device)
images = torch.load("../images/setting2_images.pt").to(device)
true_total_observed_flux = (images - 19200).sum([1,2])

N0 = 20000
kernel_num_iters = 100
catalogs_per_block = 300
setting2_std_posterior_mean_count = torch.load("./setting2_std_posterior_mean_count.pt").to(device)
setting2_std_posterior_mean_total_flux = torch.load("./setting2_std_posterior_mean_total_flux.pt").to(device)
setting2_std_log_normalizing_constant = torch.load("./setting2_std_log_normalizing_constant.pt").to(device)
setting2_std_runtime = torch.load("./setting2_std_runtime.pt").to(device)
setting2_std_num_iters = torch.load("./setting2_std_num_iters.pt").to(device)

wastefree_M = 125
setting2_wf_posterior_mean_count = torch.load("./setting2_wf_posterior_mean_count.pt").to(device)
setting2_wf_posterior_mean_total_flux = torch.load("./setting2_wf_posterior_mean_total_flux.pt").to(device)
setting2_wf_log_normalizing_constant = torch.load("./setting2_wf_log_normalizing_constant.pt").to(device)
setting2_wf_runtime = torch.load("./setting2_wf_runtime.pt").to(device)
setting2_wf_num_iters = torch.load("./setting2_wf_num_iters.pt").to(device)

## Standard results

In [3]:
print(f"RMSE (source count) across {images.shape[0]} images:", ((setting2_std_posterior_mean_count - true_counts)**2).mean().sqrt().item())
print(f"MAE (source count) across {images.shape[0]} images:", ((setting2_std_posterior_mean_count - true_counts).abs()).mean().item())
print(f"RMSE (total flux) across {images.shape[0]} images:", ((setting2_std_posterior_mean_total_flux - true_total_observed_flux)**2).mean().sqrt().item())
print(f"MAE (total flux) across {images.shape[0]} images:", ((setting2_std_posterior_mean_total_flux - true_total_observed_flux).abs()).mean().item())
print(f"correct number of sources detected in {(setting2_std_posterior_mean_count.round() == true_counts).sum()} of the {images.shape[0]} images (accuracy = {(setting2_std_posterior_mean_count.round() == true_counts).sum()/images.shape[0]})")
print(f"number of iterations: mean = {setting2_std_num_iters.mean().int()}, minimum = {setting2_std_num_iters.min().int()}, median = {setting2_std_num_iters.median().int()}, maximum = {setting2_std_num_iters.max().int()}")
print(f"runtime: mean = {setting2_std_runtime.mean().int()}, minimum = {setting2_std_runtime.min().int()}, median = {setting2_std_runtime.median().int()}, maximum = {setting2_std_runtime.max().int()}")

RMSE (source count) across 1000 images: 0.3703225553035736
MAE (source count) across 1000 images: 0.1901659369468689
RMSE (total flux) across 1000 images: 1486.6439208984375
MAE (total flux) across 1000 images: 1138.0714111328125
correct number of sources detected in 838 of the 1000 images (accuracy = 0.8379999995231628)
number of iterations: mean = 46, minimum = 32, median = 42, maximum = 174
runtime: mean = 22, minimum = 15, median = 19, maximum = 82


## Waste-free results

In [4]:
print(f"RMSE (source count) across {images.shape[0]} images:", ((setting2_wf_posterior_mean_count - true_counts)**2).mean().sqrt().item())
print(f"MAE (source count) across {images.shape[0]} images:", ((setting2_wf_posterior_mean_count - true_counts).abs()).mean().item())
print(f"RMSE (total flux) across {images.shape[0]} images:", ((setting2_wf_posterior_mean_total_flux - true_total_observed_flux)**2).mean().sqrt().item())
print(f"MAE (total flux) across {images.shape[0]} images:", ((setting2_wf_posterior_mean_total_flux - true_total_observed_flux).abs()).mean().item())
print(f"correct number of sources detected in {(setting2_wf_posterior_mean_count.round() == true_counts).sum()} of the {images.shape[0]} images (accuracy = {(setting2_wf_posterior_mean_count.round() == true_counts).sum()/images.shape[0]})")
print(f"number of iterations: mean = {setting2_wf_num_iters.mean().int()}, minimum = {setting2_wf_num_iters.min().int()}, median = {setting2_wf_num_iters.median().int()}, maximum = {setting2_wf_num_iters.max().int()}")
print(f"runtime: mean = {setting2_wf_runtime.mean().int()}, minimum = {setting2_wf_runtime.min().int()}, median = {setting2_wf_runtime.median().int()}, maximum = {setting2_wf_runtime.max().int()}")

RMSE (source count) across 1000 images: 0.39367908239364624
MAE (source count) across 1000 images: 0.2064833790063858
RMSE (total flux) across 1000 images: 1621.427490234375
MAE (total flux) across 1000 images: 1252.8985595703125
correct number of sources detected in 815 of the 1000 images (accuracy = 0.8149999976158142)
number of iterations: mean = 48, minimum = 32, median = 42, maximum = 177
runtime: mean = 48, minimum = 32, median = 42, maximum = 175


## Plots